In [1]:
import sys
sys.path.append("..")
sys.path.append("../lmmnn/")

from utils.training_functions import *
from utils.evaluation import *
from data import dataset_preprocessing

import pickle

RS = 42

C:\Users\AndrejTschalzev\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


#### Download and save data from Pargent et al. by running "data/download_pargent2022_datasets.py before running this notebook

In [2]:
dataset_name = "academic_performance"
mode="train_val_test"
RS=42
hct=10
test_ratio=0.2
val_ratio=0.1
folds=None
target = "continuous"

data_path = f"{mode}_RS{RS}_hct{hct}"
if mode == "cv":
    data_path += f"_{folds}folds"
elif mode == "train_test":
    data_path += f"_split{1-test_ratio*100}-{test_ratio*100}"
elif mode == "train_val_test":
    data_path += f"_split{round(100-(test_ratio+val_ratio)*100)}-{round(test_ratio*100)}-{round(val_ratio*100)}"


# If no data_dict for the configuration exists, run preprocessing, else load data_dict
if not os.path.exists(f"../data/prepared/{dataset_name}/"+data_path+"/data_dict.pickle"):
    dataset_preprocessing.process_dataset(dataset_name, target, mode, RS, hct, test_ratio, val_ratio, folds)
with open(f"../data/prepared/{dataset_name}/{data_path}/data_dict.pickle", 'rb') as handle:
        data_dict = pickle.load(handle)


In [3]:
z_ohe_encoded_train = data_dict["z_ohe_encoded_train"] 
z_ohe_encoded_val = data_dict["z_ohe_encoded_val"] 
z_ohe_encoded_test = data_dict["z_ohe_encoded_test"] 

z_target_encoded_train = data_dict["z_target_encoded_train"] 
z_target_encoded_val = data_dict["z_target_encoded_val"] 
z_target_encoded_test = data_dict["z_target_encoded_test"] 

X_train = data_dict["X_train"]
Z_train = data_dict["Z_train"]
y_train = data_dict["y_train"]

X_val = data_dict["X_val"]
Z_val = data_dict["Z_val"]
y_val = data_dict["y_val"]

X_test = data_dict["X_test"]
Z_test = data_dict["Z_test"]
y_test = data_dict["y_test"]

z_cols = data_dict["z_cols"]



### Correlations to target

In [4]:
pd.DataFrame(pd.concat([X_train,pd.Series(y_train,index=X_train.index,name="target")],axis=1).corr()["target"])

,target
GENDER,0.044683
INTERNET,0.188251
TV,0.135723
COMPUTER,0.125973
WASHING_MCH,0.107520
MIC_OVEN,0.171792
CAR,0.170735
DVD,0.099642
FRESH,0.038084
PHONE,0.034130


In [5]:
config = {
    # General Parameters
    "general_parameters": {
        "target": "continuous",
        "metrics": ["mse", "r2"],
        "model_name": "tabtransformer",
                             },

    # NN Parameters
    "nn_parameters": {"epochs": 200,
                      "batch_size":  100,
                      "patience": 5,
                      "stop_metric": "val_mse"},
    
    "embed_parameters":
        {"embed_dims_method": "sqrt"}
     

        }

In [6]:
train_data = [X_train, Z_train, y_train, z_ohe_encoded_train, z_target_encoded_train]
val_data = [X_val, Z_val, y_val, z_ohe_encoded_val, z_target_encoded_val]
test_data = [X_test, Z_test, y_test, z_ohe_encoded_test, z_target_encoded_test]


In [7]:
X_train_perfonly = X_train[["MAT_S11", "CR_S11", "CC_S11", "BIO_S11", "ENG_S11"]]
X_val_perfonly = X_val[["MAT_S11", "CR_S11", "CC_S11", "BIO_S11", "ENG_S11"]]
X_test_perfonly = X_test[["MAT_S11", "CR_S11", "CC_S11", "BIO_S11", "ENG_S11"]]

z_target_encoded_train_perfonly = z_target_encoded_train[["ACADEMIC_PROGRAM", "UNIVERSITY"]]
z_target_encoded_val_perfonly = z_target_encoded_val[["ACADEMIC_PROGRAM", "UNIVERSITY"]]
z_target_encoded_test_perfonly = z_target_encoded_test[["ACADEMIC_PROGRAM", "UNIVERSITY"]]

z_ohe_encoded_train_perfonly = z_ohe_encoded_train[[i for i in z_ohe_encoded_train.columns if ("ACADEMIC_PROGRAM" in i or "UNIVERSITY" in i)]]
z_ohe_encoded_val_perfonly = z_ohe_encoded_val[[i for i in z_ohe_encoded_train.columns if ("ACADEMIC_PROGRAM" in i or "UNIVERSITY" in i)]]
z_ohe_encoded_test_perfonly = z_ohe_encoded_test[[i for i in z_ohe_encoded_train.columns if ("ACADEMIC_PROGRAM" in i or "UNIVERSITY" in i)]]

Z_train_perfonly = pd.DataFrame(Z_train,columns=z_cols)[["ACADEMIC_PROGRAM", "UNIVERSITY"]].values
Z_val_perfonly = pd.DataFrame(Z_val,columns=z_cols)[["ACADEMIC_PROGRAM", "UNIVERSITY"]].values
Z_test_perfonly = pd.DataFrame(Z_test,columns=z_cols)[["ACADEMIC_PROGRAM", "UNIVERSITY"]].values

train_data_perfonly = [X_train_perfonly, Z_train_perfonly, y_train, z_ohe_encoded_train_perfonly, z_target_encoded_train_perfonly]
val_data_perfonly = [X_val_perfonly, Z_val_perfonly, y_val, z_ohe_encoded_val_perfonly, z_target_encoded_val_perfonly]
test_data_perfonly = [X_test_perfonly, Z_test_perfonly, y_test, z_ohe_encoded_test_perfonly, z_target_encoded_test_perfonly]


In [8]:
X_train_noperf = X_train.drop(["MAT_S11", "CR_S11", "CC_S11", "BIO_S11", "ENG_S11"],axis=1)
X_val_noperf = X_val.drop(["MAT_S11", "CR_S11", "CC_S11", "BIO_S11", "ENG_S11"],axis=1)
X_test_noperf = X_test.drop(["MAT_S11", "CR_S11", "CC_S11", "BIO_S11", "ENG_S11"],axis=1)

train_data_noperf = [X_train_noperf, Z_train, y_train, z_ohe_encoded_train, z_target_encoded_train]
val_data_noperf = [X_val_noperf, Z_val, y_val, z_ohe_encoded_val, z_target_encoded_val]
test_data_noperf = [X_test_noperf, Z_test, y_test, z_ohe_encoded_test, z_target_encoded_test]


## Train the models to evaluate

In [9]:
save_path = f"../results/{dataset_name}/{data_path}"
if not os.path.exists(f"{save_path}/results_dict.pickle"):
    results_dict = train_models(train_data, val_data, test_data, config, RS=RS, save_results=False,save_path=save_path)
else:
    with open(f"{save_path}/results_dict.pickle", 'rb') as handle:
        results_dict = pickle.load(handle)

results = results_dict["results"]
model_lmmnn_info = results_dict["model_lmmnn_info"]    

Load base model
Train XGBoost without z features
Train XGBoost with target encoding
Train Linear Model without z features
Train Linear Model with target encoding
Train NN without Z features
Epoch 1/200
90/90 [==============================] - 3s 10ms/step - loss: 0.7221 - r2: 0.2779 - mse: 0.7221 - val_loss: 0.4403 - val_r2: 0.5617 - val_mse: 0.4403
Epoch 2/200
90/90 [==============================] - 1s 7ms/step - loss: 0.5371 - r2: 0.4629 - mse: 0.5371 - val_loss: 0.4254 - val_r2: 0.5766 - val_mse: 0.4254
Epoch 3/200
90/90 [==============================] - 1s 7ms/step - loss: 0.4916 - r2: 0.5084 - mse: 0.4916 - val_loss: 0.3899 - val_r2: 0.6119 - val_mse: 0.3899
Epoch 4/200
90/90 [==============================] - 1s 7ms/step - loss: 0.4692 - r2: 0.5308 - mse: 0.4692 - val_loss: 0.3905 - val_r2: 0.6113 - val_mse: 0.3905
Epoch 5/200
90/90 [==============================] - 1s 7ms/step - loss: 0.4442 - r2: 0.5558 - mse: 0.4442 - val_loss: 0.3862 - val_r2: 0.6156 - val_mse: 0.3862
Epoc

90/90 [==============================] - 1s 9ms/step - loss: 0.3660 - r2: 0.6340 - mse: 0.3660 - val_loss: 0.3974 - val_r2: 0.6045 - val_mse: 0.3974
Epoch 11/200
90/90 [==============================] - 1s 9ms/step - loss: 0.3507 - r2: 0.6493 - mse: 0.3507 - val_loss: 0.3873 - val_r2: 0.6145 - val_mse: 0.3873
Epoch 12/200
90/90 [==============================] - 1s 8ms/step - loss: 0.3365 - r2: 0.6635 - mse: 0.3365 - val_loss: 0.4023 - val_r2: 0.5996 - val_mse: 0.4023
Epoch 13/200
90/90 [==============================] - 1s 9ms/step - loss: 0.3281 - r2: 0.6719 - mse: 0.3281 - val_loss: 0.3830 - val_r2: 0.6188 - val_mse: 0.3830
Epoch 14/200
90/90 [==============================] - 1s 9ms/step - loss: 0.3222 - r2: 0.6778 - mse: 0.3222 - val_loss: 0.4068 - val_r2: 0.5951 - val_mse: 0.4068
Epoch 15/200
90/90 [==============================] - 1s 9ms/step - loss: 0.3119 - r2: 0.6881 - mse: 0.3119 - val_loss: 0.4269 - val_r2: 0.5751 - val_mse: 0.4269
Epoch 16/200
90/90 [=====================

In [11]:
save_path = f"../results/{dataset_name}/{data_path}"
if not os.path.exists(f"{save_path}/results_dict_perfonly.pickle"):
    results_dict_perfonly = train_models(train_data_perfonly, val_data_perfonly, test_data_perfonly, config, RS=RS, save_results=False,save_path=save_path)
else:
    with open(f"{save_path}/results_dict_perfonly.pickle", 'rb') as handle:
        results_dict_perfonly = pickle.load(handle)

results_perfonly = results_dict_perfonly["results"]
model_lmmnn_info_perfonly = results_dict_perfonly["model_lmmnn_info"]    

Load base model
Train XGBoost without z features
Train XGBoost with target encoding
Train Linear Model without z features
Train Linear Model with target encoding
Train NN without Z features
Epoch 1/200
90/90 [==============================] - 2s 10ms/step - loss: 1.4204 - r2: -0.4204 - mse: 1.4204 - val_loss: 1.0445 - val_r2: -0.0396 - val_mse: 1.0445
Epoch 2/200
90/90 [==============================] - 1s 6ms/step - loss: 1.1821 - r2: -0.1821 - mse: 1.1821 - val_loss: 1.0322 - val_r2: -0.0274 - val_mse: 1.0322
Epoch 3/200
90/90 [==============================] - 1s 6ms/step - loss: 1.1163 - r2: -0.1163 - mse: 1.1163 - val_loss: 1.0235 - val_r2: -0.0187 - val_mse: 1.0235
Epoch 4/200
90/90 [==============================] - 1s 6ms/step - loss: 1.0876 - r2: -0.0876 - mse: 1.0876 - val_loss: 1.0228 - val_r2: -0.0180 - val_mse: 1.0228
Epoch 5/200
90/90 [==============================] - 1s 6ms/step - loss: 1.0668 - r2: -0.0668 - mse: 1.0668 - val_loss: 1.0157 - val_r2: -0.0109 - val_mse: 1

90/90 [==============================] - 1s 6ms/step - loss: 0.4903 - r2: 0.5097 - mse: 0.4903 - val_loss: 0.4593 - val_r2: 0.5429 - val_mse: 0.4593
Epoch 25/200
90/90 [==============================] - 1s 7ms/step - loss: 0.4891 - r2: 0.5109 - mse: 0.4891 - val_loss: 0.4578 - val_r2: 0.5443 - val_mse: 0.4578
Epoch 26/200
90/90 [==============================] - 1s 8ms/step - loss: 0.4886 - r2: 0.5114 - mse: 0.4886 - val_loss: 0.4559 - val_r2: 0.5463 - val_mse: 0.4559
Epoch 27/200
90/90 [==============================] - 1s 6ms/step - loss: 0.4912 - r2: 0.5088 - mse: 0.4912 - val_loss: 0.4564 - val_r2: 0.5458 - val_mse: 0.4564
Epoch 28/200
90/90 [==============================] - 1s 6ms/step - loss: 0.4870 - r2: 0.5130 - mse: 0.4870 - val_loss: 0.4554 - val_r2: 0.5468 - val_mse: 0.4554
Epoch 29/200
90/90 [==============================] - 1s 7ms/step - loss: 0.4882 - r2: 0.5118 - mse: 0.4882 - val_loss: 0.4566 - val_r2: 0.5456 - val_mse: 0.4566
Epoch 30/200
90/90 [=====================

90/90 [==============================] - 1s 10ms/step - loss: 0.3892 - r2: 0.6108 - mse: 0.3892 - val_loss: 0.3775 - val_r2: 0.6242 - val_mse: 0.3775
Epoch 19/200
90/90 [==============================] - 1s 11ms/step - loss: 0.3854 - r2: 0.6146 - mse: 0.3854 - val_loss: 0.3717 - val_r2: 0.6300 - val_mse: 0.3717
Epoch 20/200
90/90 [==============================] - 1s 10ms/step - loss: 0.3858 - r2: 0.6142 - mse: 0.3858 - val_loss: 0.3800 - val_r2: 0.6218 - val_mse: 0.3800
Epoch 21/200
90/90 [==============================] - 1s 10ms/step - loss: 0.3816 - r2: 0.6184 - mse: 0.3816 - val_loss: 0.3698 - val_r2: 0.6319 - val_mse: 0.3698
Epoch 22/200
90/90 [==============================] - 1s 11ms/step - loss: 0.3779 - r2: 0.6221 - mse: 0.3779 - val_loss: 0.3699 - val_r2: 0.6318 - val_mse: 0.3699
Epoch 23/200
90/90 [==============================] - 1s 11ms/step - loss: 0.3786 - r2: 0.6214 - mse: 0.3786 - val_loss: 0.3729 - val_r2: 0.6288 - val_mse: 0.3729
Epoch 24/200
90/90 [===============

In [12]:
save_path = f"../results/{dataset_name}/{data_path}"
if not os.path.exists(f"{save_path}/results_dict_noperf.pickle"):
    results_dict_noperf = train_models(train_data_noperf, val_data_noperf, test_data_noperf, config, RS=RS, save_results=False,save_path=save_path)
else:
    with open(f"{save_path}/results_dict.pickle_noperf", 'rb') as handle:
        results_dict_noperf = pickle.load(handle)

results_noperf = results_dict_noperf["results"]
model_lmmnn_info_noperf = results_dict_noperf["model_lmmnn_info"]    

Load base model
Train XGBoost without z features
Train XGBoost with target encoding
Train Linear Model without z features
Train Linear Model with target encoding
Train NN without Z features
Epoch 1/200
90/90 [==============================] - 2s 9ms/step - loss: 1.1970 - r2: -0.1970 - mse: 1.1970 - val_loss: 0.8516 - val_r2: 0.1524 - val_mse: 0.8516
Epoch 2/200
90/90 [==============================] - 1s 6ms/step - loss: 0.9688 - r2: 0.0312 - mse: 0.9688 - val_loss: 0.8305 - val_r2: 0.1734 - val_mse: 0.8305
Epoch 3/200
90/90 [==============================] - 1s 7ms/step - loss: 0.9281 - r2: 0.0719 - mse: 0.9281 - val_loss: 0.8410 - val_r2: 0.1630 - val_mse: 0.8410
Epoch 4/200
90/90 [==============================] - 1s 7ms/step - loss: 0.8942 - r2: 0.1058 - mse: 0.8942 - val_loss: 0.8159 - val_r2: 0.1880 - val_mse: 0.8159
Epoch 5/200
90/90 [==============================] - 1s 8ms/step - loss: 0.8688 - r2: 0.1312 - mse: 0.8688 - val_loss: 0.8212 - val_r2: 0.1827 - val_mse: 0.8212
Epoc

90/90 [==============================] - 1s 10ms/step - loss: 0.5027 - r2: 0.4973 - mse: 0.5027 - val_loss: 0.6614 - val_r2: 0.3417 - val_mse: 0.6614
Epoch 13/200
90/90 [==============================] - 1s 8ms/step - loss: 0.4893 - r2: 0.5107 - mse: 0.4893 - val_loss: 0.6669 - val_r2: 0.3362 - val_mse: 0.6669
Epoch 14/200
90/90 [==============================] - 1s 9ms/step - loss: 0.4727 - r2: 0.5273 - mse: 0.4727 - val_loss: 0.6899 - val_r2: 0.3134 - val_mse: 0.6899
Epoch 15/200
90/90 [==============================] - 1s 9ms/step - loss: 0.4747 - r2: 0.5253 - mse: 0.4747 - val_loss: 0.6755 - val_r2: 0.3276 - val_mse: 0.6755
Epoch 16/200
25/25 [==============================] - 0s 1ms/step
Train NN with Embeddings
Epoch 1/200
90/90 [==============================] - 3s 21ms/step - loss: 1.3888 - r2: -0.3188 - mse: 1.3194 - val_loss: 0.9080 - val_r2: 0.0962 - val_mse: 0.9080
Epoch 2/200
90/90 [==============================] - 2s 17ms/step - loss: 0.8421 - r2: 0.1579 - mse: 0.8421 - 

## Evaluation

### Performance on whole data

In [13]:
models_use = ["LR", "XGB", "LR_te", "XGB_te", "NN", "NN_te", "NN_ohe", "NN_embed", "LMMNN"]

In [14]:
results_df = pd.DataFrame(results).transpose().loc[models_use].sort_values("MSE Test",ascending=False).round(4)


In [15]:
results_df[["MSE Train", "R2 Train", "MSE Test", "R2 Test"]].style.highlight_min(subset=["MSE Train", "MSE Test"], color = 'lightgreen', axis = 0).highlight_max(subset=["R2 Train", "R2 Test"], color = 'lightgreen', axis = 0)

,MSE Train,R2 Train,MSE Test,R2 Test
NN_embed,0.220900,0.779100,0.449000,0.538400
XGB_te,0.114400,0.885600,0.447600,0.539800
NN_ohe,0.237300,0.762700,0.433400,0.554400
XGB,0.163500,0.836500,0.430500,0.557400
NN_te,0.327800,0.672200,0.402200,0.586500
LR_te,0.341900,0.658100,0.400700,0.588000
LMMNN,0.268400,0.731600,0.391900,0.597100
NN,0.367300,0.632700,0.388000,0.601100
LR,0.382300,0.617700,0.386600,0.602500


#### Learned variance parameters

In [16]:
pd.DataFrame(model_lmmnn_info["sigmas"][1],index=data_dict["z_cols"]).transpose().round(3)

,EDU_FATHER,EDU_MOTHER,OCC_FATHER,OCC_MOTHER,PEOPLE_HOUSE,SCHOOL_NAME,UNIVERSITY,ACADEMIC_PROGRAM
0,0.082,0.003,0.0,0.007,0.002,0.074,0.023,0.004


### Modeling results only with previous performance

In [18]:
results_df_perfonly = pd.DataFrame(results_perfonly).transpose().loc[models_use].sort_values("MSE Test",ascending=False).round(4)


In [19]:
results_df_perfonly[["MSE Train", "R2 Train", "MSE Test", "R2 Test"]].style.highlight_min(subset=["MSE Train", "MSE Test"], color = 'lightgreen', axis = 0).highlight_max(subset=["R2 Train", "R2 Test"], color = 'lightgreen', axis = 0)

,MSE Train,R2 Train,MSE Test,R2 Test
NN,0.992200,0.007800,0.972700,-0.000000
LMMNN,0.590500,0.409500,0.600300,0.382800
NN_te,0.469300,0.530700,0.476900,0.509700
XGB,0.212100,0.787900,0.434600,0.553200
XGB_te,0.164700,0.835300,0.418400,0.569900
LR,0.390300,0.609700,0.391100,0.597900
NN_ohe,0.334300,0.665700,0.390300,0.598700
NN_embed,0.359800,0.640200,0.381100,0.608200
LR_te,0.370700,0.629300,0.376600,0.612800


In [23]:
pd.DataFrame(model_lmmnn_info_perfonly["sigmas"][1],index=z_cols[-2:]).transpose().round(3)

,UNIVERSITY,ACADEMIC_PROGRAM
0,0.096,0.284


### Modeling results without previous performance

In [24]:
results_df_noperf = pd.DataFrame(results_noperf).transpose().loc[models_use].sort_values("MSE Test",ascending=False).round(4)


In [25]:
results_df_noperf[["MSE Train", "R2 Train", "MSE Test", "R2 Test"]].style.highlight_min(subset=["MSE Train", "MSE Test"], color = 'lightgreen', axis = 0).highlight_max(subset=["R2 Train", "R2 Test"], color = 'lightgreen', axis = 0)

,MSE Train,R2 Train,MSE Test,R2 Test
XGB,0.535000,0.465000,0.858900,0.117000
LR,0.804700,0.195300,0.797000,0.180700
NN,0.771500,0.228500,0.793600,0.184200
NN_embed,0.382100,0.617900,0.699700,0.280600
NN_ohe,0.382300,0.617700,0.677700,0.303300
XGB_te,0.201600,0.798400,0.668100,0.313200
NN_te,0.495900,0.504100,0.640000,0.342000
LR_te,0.508300,0.491700,0.634800,0.347400
LMMNN,0.449500,0.550500,0.606900,0.376000


In [26]:
pd.DataFrame(model_lmmnn_info_noperf["sigmas"][1],index=data_dict["z_cols"]).transpose().round(3)

,EDU_FATHER,EDU_MOTHER,OCC_FATHER,OCC_MOTHER,PEOPLE_HOUSE,SCHOOL_NAME,UNIVERSITY,ACADEMIC_PROGRAM
0,0.053,0.007,0.0,0.006,0.0,0.105,0.21,0.029
